In [1]:
# Calculate Watt Spectra weighting given a set of input bins
import csv    # Library to read & process csv-formatted files
import sys    # Library to parse command-line arguments
import os
from math import sqrt, sinh, pi, exp
import numpy as np

rootdir=os.path.abspath(os.getcwd())

# Optional Paramater set - ref=Verbeke2014
# Assume avg fission energy of 1 MeV for benchmark of the Watt induced FP distibution
a0=0.920108
a1=-0.00936909
a2=7.32627e-05
b=1.0
En=1.0
fname = rootdir+"/Bins.csv"   # Bin structure input file

# Read in the bin structure from file
try: 
    ifile = open(fname, 'r') 
    ifile.next() # skip the first header line

    # Store the spectrum
    bins=[]
    for line in ifile:
        split_list=line.split(',')
        bins.append(float(split_list[2].strip()))

    # Close the file
    ifile.close()
except IOError as e:
    print "I/O error({0}): {1}".format(e.errno, e.strerror)
    print "File not found was: {0}".format(fname)
    
# Calculate the Watt spectra
def Watt(E,a0,a1,a2,b,E_inc):
    a=a0+a1*E_inc+a2*E_inc**2
    c=sqrt(pi*b/(4*a))*exp(b/(4*a))/a
    w=c*exp(-a*E)*sinh(sqrt(b*E))
    return w

watt=[]
for e in bins:
    watt.append(Watt(e,a0,a1,a2,b,En))
    
np.asarray(watt)
watt=watt/np.sum(watt)

In [25]:
y={}
err={}

w=[]
fy=[]

one=0
two=0

# Use weighted watt spectra to generate mass yield curves (sum yields)
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file=="GEF.dat":
            print subdir, file
            weight = 0
            one+=1
            try: 
                ifile = open(subdir+'/'+file, 'r') 
                for line in ifile:
                    if line[:7] == ' formed':
                        split_list=line.split(' ')
                        for i in range(0,len(bins)):
                            if bins[i] == float(split_list[8].strip()):
                                two+=1
                                weight=watt[i]
                                print "Energy =", bins[i]
                                print "Fractional weight=",weight
                    if line[:27] =='--- Mass-yield distribution':
                        for i in range(0,6):
                            line=ifile.next()
                        while line[:22] != '        </Mass_yields>':
                            split_list=line.split()
                            if not split_list[0] in y.keys():  # this is the first time we've encountered this mass A
                            # let's initialize our dictionaries
                                y[split_list[0]] = float(split_list[2])*weight
                                err[split_list[0]] = (float(split_list[3])*weight)**2
                            else:  # we've encoutered this mass before, no need to initialize, just add to it
                                y[split_list[0]] += float(split_list[2])*weight
                                err[split_list[0]] += (float(split_list[3])*weight)**2
                                
                            if int(split_list[0])==140:
                                w.append(weight)
                                fy.append(float(split_list[2]))
                            line=ifile.next()
                # Close the file
                ifile.close()
            except IOError as e:
                print "I/O error({0}): {1}".format(e.errno, e.strerror)
                print "File not found was: {0}".format(fname)
print y
print y.keys()
print one
print two

/home/pyne-user/Dropbox/UCB/Research/ETAs/Scoping_Calcs/FP_Distro/Validation/Watt/Z92_A236_n_E0.0434MeV GEF.dat
Energy = 0.0434
Fractional weight= 0.0284060011941
/home/pyne-user/Dropbox/UCB/Research/ETAs/Scoping_Calcs/FP_Distro/Validation/Watt/Z92_A236_n_E5.67MeV GEF.dat
Energy = 5.67
Fractional weight= 0.00431758611489
/home/pyne-user/Dropbox/UCB/Research/ETAs/Scoping_Calcs/FP_Distro/Validation/Watt/Z92_A236_n_E2.07e-007MeV GEF.dat
Energy = 2.07e-07
Fractional weight= 6.40738487576e-05
/home/pyne-user/Dropbox/UCB/Research/ETAs/Scoping_Calcs/FP_Distro/Validation/Watt/Z92_A236_n_E6.87e-006MeV GEF.dat
Energy = 6.87e-06
Fractional weight= 0.000369123635219
/home/pyne-user/Dropbox/UCB/Research/ETAs/Scoping_Calcs/FP_Distro/Validation/Watt/Z92_A236_n_E0.0818MeV GEF.dat
Energy = 0.0818
Fractional weight= 0.0378983509327
/home/pyne-user/Dropbox/UCB/Research/ETAs/Scoping_Calcs/FP_Distro/Validation/Watt/Z92_A236_n_E0.691MeV GEF.dat
Energy = 0.691
Fractional weight= 0.0698255804652
/home/pyne-us

In [13]:
print w
print fy

[0.028406001194131756, 0.0043175861148899873, 0, 8.6010752193189593e-06, 8.6010752193189593e-06, 8.6010752193189593e-06, 8.6010752193189593e-06, 6.4073848757584365e-05, 0.00020359549945658432, 0.00020359549945658432, 0.000369123635218515, 0.037898350932717907, 0.069825580465170525, 0.036570384598068231, 0.069636964198285245, 0.0076424885486602694, 8.6010752193189593e-06, 0.066044469121440935, 2.9623889347558109e-05, 2.9286530181199713e-07, 0.05444928369513085, 0.00062822586487117451, 0.00020359549945658432, 0.010338964934742428, 0.00094097603420496096, 0.046655114145691318, 0.042514914998281493, 0.0042431059528894692, 0.0011370994057547706, 0.0005149780206111105, 0.01795974718152853, 0.0018194422715777023, 0.061172509495373582, 5.4681065841581069e-05, 5.3914788996907653e-06, 0.02366305748109148, 1.9491349022988695e-06, 0.029972333736203054, 0.062116595384727677, 0.00026364120285573536, 0.002915987836714715, 0.011588123218504635, 0.052985087134859957, 1.5989362406610427e-05, 0.067751121

In [26]:
# Output the result
outfn="235U_watt_sum_fy.csv"

def writeout(outfn,outl):  # function to write our output in csv format
    ofile = open(outfn,"w")

    for out in outl:
        ofile.write(str(out[0])
                    + "," + str(out[1])
                    + "," + str(out[2])
                    + "," + str(sqrt(out[3]))
                    + "," + str(out[4])
                    + "\n")

    ofile.close()

outl=[]
for A in y.keys():
    outl.append((A,0,y[A],err[A]," "))
writeout(outfn,outl)